In [1]:
%matplotlib widget
import numpy as np
import os
import h5py
import subprocess

print(os.getpid())
%cd ../
import pylib.mix as mix
%cd "./jupyter-notebooks/QSVT-QSP-angles-for-matlab-c#/Hamiltonian/QSP/"

29394
/media/work/docs/codes/QuCF/scripts-py
/media/work/docs/codes/QuCF/scripts-py/jupyter-notebooks/QSVT-QSP-angles-for-matlab-c#/Hamiltonian/QSP


In [2]:
mix.reload_module(mix)

# -----------------------------------------------------------
def find_exp(number) -> int:
    base10 = np.log10(abs(number))

    temp = np.floor(base10)
    flag_neg = False
    if temp < 0:
        flag_neg = True

    return abs(temp), flag_neg
# -----------------------------------------------------------
def txt_to_hdf5(old_filename, new_filename):
    function_type_res = "QSP-ham"
    rescaling_factor_res = 1.
    parity_res = -1

    # --- read the .angles (text) file ---
    print("read angles from:\n " + old_filename)
    with open(old_filename) as f:
        param_res = float(f.readline()[:-1]) 
        eps_res   = float(f.readline()[:-1]) 
        Na        = int(f.readline()[:-1]) 
        angles = np.zeros(Na)
        for ii in range(Na):
            angles[ii] = float(f.readline())
            
    print()
    print("function type: \t\t{:s}".format(function_type_res))    
    print("rescaling factor: \t{:0.3f}".format(rescaling_factor_res))
    print("parity: \t\t{:d}".format(parity_res))
    print("parameter: \t\t{:0.3e}".format(param_res))
    print("error: \t\t\t{:0.3e}".format(eps_res))
    print("N-angles: \t\t{:d}".format(Na))
    print()       

    # --- write the .hdf5 file ---
    print("write angles to:\n " + new_filename)
    with h5py.File(new_filename, "w") as f:
        grp = f.create_group("basic")
        
        grp.create_dataset('polynomial_type',  data=function_type_res)
        grp.create_dataset('rescaling_factor', data=float(rescaling_factor_res))
        grp.create_dataset('parity',           data=parity_res)
        grp.create_dataset('eps',              data=eps_res)
        grp.create_dataset('par',              data=param_res)

        grp = f.create_group("angles")
        grp.create_dataset('QSP-ham',  data = angles)
    return

In [5]:
# ----------------------------------------------------------
# --- Specify QSP parameters ---
# ----------------------------------------------------------
t_moments = [7.467004919912e+04/12.]



Nt = len(t_moments)
eps_QSP_array = [1e-16] * Nt

case_format = 1

# --- Form the name of the output name ---
file_names = [None] * Nt
print("--- Chosen QSP parameters ---")
if case_format == 1:
    for it in range(Nt):
        eps_QSP = eps_QSP_array[it]
        t1 = t_moments[it]
        exp_t1, flag_neg = find_exp(t1)

        str_exp = "p"
        if flag_neg:
            str_exp = "m"

        if flag_neg:
            temp_int = int(t1 * 10**(exp_t1+3))
        else:
            temp_int = int(t1 / 10**(exp_t1-3))

        file_names[it] = "QSP_t{:d}{:s}{:d}_eps{:d}".format(
            temp_int, 
            str_exp,
            int(exp_t1),
            int(abs(np.round(np.log10(eps_QSP))))
        )

        print("t, eps: {:0.12e}, {:0.3e}".format(t1, eps_QSP))
        print("file name: {:s}".format(file_names[it]))
        print()

--- Chosen QSP parameters ---
t, eps: 6.222504099927e+03, 1.000e-16
file name: QSP_t6222p3_eps16



In [4]:
# ------------------------------------------------
# --- Compute QSP angles ---
# ------------------------------------------------
line_code = "/media/work/docs/codes/progs/Q#F#/test-qsp/bin/Debug/netcoreapp3.1/qsp"
for it in range(Nt):
    eps_QSP = eps_QSP_array[it]
    t1 = t_moments[it]

    line_cmd = "{:s} {:0.12e} {:0.3e} {:s} {:s}".format(
        line_code, t1, eps_QSP, "./", file_names[it]
    )
    subprocess.run(line_cmd, shell = True, executable="/bin/bash")

    # from the .txt file to .hdf5 file:
    txt_to_hdf5(file_names[it], file_names[it] + ".hdf5")

    # remove the .txt file:
    line_cmd = "rm {:s}".format(file_names[it])
    subprocess.run(line_cmd, shell = True, executable="/bin/bash")

Start calculating...
Chosen tau = 6222.504099926
Chosen eps = 1E-16
polyA = { Denominator = 1180591620717411303424
  Numerator =
             1z^-6460 + 2z^-6458 + 3z^-6456 + 6z^-6454 + 11z^-6452 + 19z^-6450 + 32z^-6448 + 55z^-6446 + 95z^-6444 + 162z^-6442 + 276z^-6440 + 468z^-6438 + 793z^-6436 + 1339z^-6434 + 2255z^-6432 + 3789z^-6430 + 6349z^-6428 + 10614z^-6426 + 17699z^-6424 + 29441z^-6422 + 48851z^-6420 + 80853z^-6418 + 133483z^-6416 + 219814z^-6414 + 361060z^-6412 + 591552z^-6410 + 966694z^-6408 + 1575659z^-6406 + 2561575z^-6404 + 4153526z^-6402 + 6717161z^-6400 + 10834461z^-6398 + 17429109z^-6396 + 27962943z^-6394 + 44742867z^-6392 + 71398853z^-6390 + 113626166z^-6388 + 180334246z^-6386 + 285419563z^-6384 + 450492696z^-6382 + 709059926z^-6380 + 1112914632z^-6378 + 1741874678z^-6376 + 2718569119z^-6374 + 4230818630z^-6372 + 6565396794z^-6370 + 10158788652z^-6368 + 15673245044z^-6366 + 24110348493z^-6364 + 36980004776z^-6362 + 56551030193z^-6360 + 86221416616z^-6358 + 131063472095